In [1]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
_query = """
SELECT
  trim('"' FROM block::text )::int as block,
  trim('"' FROM min::text )::int as min,
  s.holding,
  s.acks,
  s.msgqueue,
  s.inmsgqueue,
  s.apiqueue,
  s.ackqueue,
  s.timermsg,
  ts::text::time,
  s.run
FROM (SELECT
  --mt     | "LIST_SIZES Holding: %v, Acks: %v, MsgQueue: %v, InMsgQueue: %v, APIQueue: %v, AckQueue: %v, TimerMsg: %v "
  run,
  e->'log'->'height' as block,
  e->'log'->'min' as min,
  e->'log'->'event'->0 as holding,
  e->'log'->'event'->1 as acks,
  e->'log'->'event'->2 as msgqueue,
  e->'log'->'event'->3 as inmsgqueue,
  e->'log'->'event'->4 as apiqueue,
  e->'log'->'event'->5 as ackqueue,
  e->'log'->'event'->6 as timermsg,
  e->'log'->'event'->7 as fmt,
  e->'ts' as ts
  --e as l
FROM
  logs
) as s

WHERE
  s.fmt::varchar like '%LIST_SIZES%'
"""

_order = """
ORDER BY ts;
"""


def query(i):
    return _query + ' AND run = %s ' % i + _order

connection = pg.connect("host='localdb' dbname=load user=load password='load'")


In [3]:
log_runs = pd.read_sql_query('SELECT * from log_runs;', con=connection)
log_runs

,id,label,ts
0,1,v5_test_LLLLLLLFF_d100,2019-01-31 17:07:33.116239
1,2,v5_test_LLLLLLLF_d100,2019-01-31 17:08:19.244705
2,3,v6_test_LLLLLLLFF_d100,2019-01-31 17:09:05.469478
3,4,v6_test_LLLLLLLF_d100,2019-01-31 17:09:52.358996


In [4]:
import plotly.plotly as py
import plotly.graph_objs as go

data = []
for i, n in log_runs['label'].items():
    r = log_runs['id'][i]
    df = pd.read_sql_query(query(r), con=connection)
    #data.append(
    #    go.Scatter(x=df['ts'], y=df['holding'], name=n.replace('test_','')+'holding')
    #)
    data.append(
        go.Scatter(x=df['ts'], y=df['inmsgqueue'], name=n.replace('test_','')+'inmsg')
    )

py.iplot(data, filename='WIP-queues')